# Basic Transformations

As part of this module we will learn following things.
* Overview of Basic Transformations
* Starting Spark Context
* Overview of Filtering
* Overview of Aggregations
* Overview of Sorting
* Problems and Solutions

## Overview of Basic Transformations

Let us define problem statements to learn more about Data Frame APIs. We will try to cover filtering, aggregations and sorting as part of solutions for these problem statements.
* Get total number of flights as well as number of flights which are delayed in departure and number of flights delayed in arrival. 
 * Output should contain 3 columns - **FlightCount**, **DepDelayedCount**, **ArrDelayedCount**
* Get number of flights which are delayed in departure and number of flights delayed in arrival for each day along with number of flights departed for each day. 
 * Output should contain 4 columns - **FlightDate**, **FlightCount**, **DepDelayedCount**, **ArrDelayedCount**
 * **FlightDate** should be of **YYYY-MM-dd** format.
 * Data should be **sorted** in ascending order by **flightDate**
* Get all the flights which are departed late but arrived early (**IsArrDelayed is NO**).
 * Output should contain - **FlightCRSDepTime**, **UniqueCarrier**, **FlightNum**, **Origin**, **Dest**, **DepDelay**, **ArrDelay**
 * **FlightCRSDepTime** need to be computed using **Year**, **Month**, **DayOfMonth**, **CRSDepTime**
 * **FlightCRSDepTime** should be displayed using **YYYY-MM-dd HH:mm** format.
 * Output should be sorted by **FlightCRSDepTime** and then by the difference between **DepDelay** and **ArrDelay**
 * Also get the count of such flights

## Starting Spark Context

Let us start spark context for this Notebook so that we can execute the code provided.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    appName('Basic Transformations'). \
    master('yarn'). \
    getOrCreate()

 
## Overview of Filtering
Let us understand few important details related to filtering before we get into the solution
* Filtering can be done either by using `filter` or `where`. These are like synonyms to each other.
* When it comes to the condition, we can either pass it in **SQL Style** or **Data Frame Style**.
* Example for SQL Style - `airlines.filter("IsArrDelayed = 'YES'").show() or airlines.where("IsArrDelayed = 'YES'").show()`
* Example for Data Frame Style - `airlines.filter(airlines["IsArrDelayed"] == 'YES').show()` or `airlines.filter(airlines.IsArrDelayed == 'YES').show()`. We can also use where instead of filter.
* Here are the other operations we can perform to filter the data - `!=`, `>`, `<`, `>=`, `<=`, `LIKE`, `BETWEEN` with `AND`
* If we have to validate against multiple columns then we need to use boolean operations such as `AND` and `OR`.
* If we have to compare each column value with multiple values then we can use the `IN` operator.

### Tasks

Let us perform some tasks to understand filtering in detail. Solve all the problems by passing  conditions using both SQL Style as well as API Style.

* Read the data for the month of 2008 January.

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
    read. \
    parquet(airlines_path)

In [ ]:
airlines.printSchema()

* Get count of flights which are departed late at origin and reach destination early or on time.


In [ ]:
airlines. \
    filter("IsDepDelayed = 'YES' AND IsArrDelayed = 'NO'"). \
    count()

* API Style

In [ ]:
from pyspark.sql.functions import col

In [ ]:
airlines. \
    filter((col("IsDepDelayed") == "YES") & 
           (col("IsArrDelayed") == "NO")
          ). \
    count()

In [ ]:
airlines. \
    filter((airlines["IsDepDelayed"] == "YES") & 
           (airlines.IsArrDelayed == "NO")
          ). \
    count()

* Get count of flights which are departed late from origin by more than 60 minutes.


In [ ]:
airlines. \
    filter("DepDelay > 60"). \
    count()


* API Style

In [ ]:
from pyspark.sql.functions import col

airlines. \
    filter(col("DepDelay") > 60). \
    count()

* Get count of flights which are departed early or on time but arrive late by at least 15 minutes.


In [ ]:
airlines. \
    filter("IsDepDelayed = 'NO' AND ArrDelay >= 15"). \
    count()

* API Style

In [ ]:
from pyspark.sql.functions import col

airlines. \
    filter((col("IsDepDelayed") == "NO") & (col("ArrDelay") >= 15)). \
    count()

* Get count of flights departed from following major airports - ORD, DFW, ATL, LAX, SFO.

In [ ]:
airlines. \
    filter("Origin IN ('ORD', 'DFW', 'ATL', 'LAX', 'SFO')"). \
    count()

In [ ]:
airlines.count()

* API Style

In [ ]:
from pyspark.sql.functions import col
c = col('x')
help(c.isin)

In [ ]:
from pyspark.sql.functions import col

airlines. \
    filter(col("Origin").isin("ORD", "DFW", "ATL", "LAX", "SFO")). \
    count()

* Add a column FlightDate by using Year, Month and DayOfMonth. Format should be **yyyyMMdd**.


In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
                concat(col("Year"),
                       lpad(col("Month"), 2, "0"),
                       lpad(col("DayOfMonth"), 2, "0")
                      )
              ). \
    show()

* Get count of flights departed late between 2008 January 1st to January 9th using FlightDate.


In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("IsDepDelayed = 'YES' AND FlightDate LIKE '2008010%'"). \
    count()

In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           IsDepDelayed = 'YES' AND 
           FlightDate BETWEEN 20080101 AND 20080109
          """). \
    count()

* API Style

In [ ]:
from pyspark.sql.functions import col
c = col('x')
help(c.like)

In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") & 
           (col("FlightDate").like("2008010%"))
          ). \
    count()

In [ ]:
from pyspark.sql.functions import col
c = col('x')
help(c.between)

In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") & 
           (col("FlightDate").between("20080101", "20080109"))
          ). \
    count()

* Get number of flights departed late on Sundays.

In [ ]:
l = [('X',)]
df = spark.createDataFrame(l, "dummy STRING")

In [ ]:
from pyspark.sql.functions import current_date
df.select(current_date()).show()

In [ ]:
from pyspark.sql.functions import date_format

df.select(current_date(), date_format(current_date(), 'EE')).show()

In [ ]:
from pyspark.sql.functions import col, concat, lpad

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter("""
           IsDepDelayed = 'YES' AND 
           date_format(to_date(FlightDate, 'yyyyMMdd'), 'EEEE') = 'Sunday'
           """). \
    count()

* API Style

In [ ]:
from pyspark.sql.functions import col, concat, lpad, date_format, to_date

airlines. \
    withColumn("FlightDate",
               concat(col("Year"),
                      lpad(col("Month"), 2, "0"),
                      lpad(col("DayOfMonth"), 2, "0")
                     )
              ). \
    filter((col("IsDepDelayed") == "YES") &
           (date_format(
               to_date("FlightDate", "yyyyMMdd"), "EEEE"
           ) == "Sunday")
          ). \
    count()

## Overview of Aggregations

Let us go through the details related to aggregation using Spark.

* We can perform total aggregations directly on Dataframe or we can perform aggregations after grouping by a key(s).
* Here are the APIs which we typically use to group the data using a key.
 * groupBy
 * rollup
 * cube
* Here are the functions which we typically use to perform aggregations.
 * count
 * sum, avg
 * min, max
* If we want to provide aliases to the aggregated fields then we have to use `agg` after `groupBy`.

## Overview of Sorting

Let us understand how to sort the data in a Data Frame.
* We can use `orderBy` or `sort` to sort the data.
* We can perform composite sorting by passing multiple columns or expressions.
* By default data is sorted in ascending order, we can change it to descending by applying `desc()` function on the column or expression.

## Solutions - Problem 1
Get total number of flights as well as number of flights which are delayed in departure and number of flights delayed in arrival. 
* Output should contain 3 columns - **FlightCount**, **DepDelayedCount**, **ArrDelayedCount**

### Reading airlines data

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

airlines = spark. \
    read. \
    parquet(airlines_path)

airlines.printSchema()

### Get flights with delayed arrival

In [ ]:
# SQL Style
airlines.filter("IsArrDelayed = 'YES'").show()

In [ ]:
# Data Frame Style
airlines.filter(airlines["IsArrDelayed"] == 'YES').show()

In [ ]:
airlines.filter(airlines.IsArrDelayed == 'YES').show()

### Get delayed counts

In [ ]:
## Departure Delayed Count
airlines. \
    filter(airlines.IsDepDelayed == "YES"). \
    count()

In [ ]:
## Arrival Delayed Count
airlines. \
    filter(airlines.IsArrDelayed == "YES"). \
    count()

In [ ]:
airlines. \
    filter("IsDepDelayed = 'YES' OR IsArrDelayed = 'YES'"). \
    select('Year', 'Month', 'DayOfMonth', 
           'FlightNum', 'IsDepDelayed', 'IsArrDelayed'
          ). \
    show()

In [ ]:
## Both Departure Delayed and Arrival Delayed
from pyspark.sql.functions import col, lit, count, sum, expr
airlines. \
    agg(count(lit(1)).alias("FlightCount"),
        sum(expr("CASE WHEN IsDepDelayed = 'YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed = 'YES' THEN 1 ELSE 0 END")).alias("ArrDelayedCount")
       ). \
    show()

## Solutions - Problem 2

Get number of flights which are delayed in departure and number of flights delayed in arrival for each day along with number of flights departed for each day. 

* Output should contain 4 columns - **FlightDate**, **FlightCount**, **DepDelayedCount**, **ArrDelayedCount**
* **FlightDate** should be of **YYYY-MM-dd** format.
*   Data should be **sorted** in ascending order by **flightDate**

### Grouping Data by Flight Date


In [ ]:
from pyspark.sql.functions import lit, concat, lpad
airlines. \
  groupBy(concat("Year", lit("-"), 
                 lpad("Month", 2, "0"), lit("-"), 
                 lpad("DayOfMonth", 2, "0")).
          alias("FlightDate"))

### Getting Counts by FlightDate

In [ ]:
from pyspark.sql.functions import lit, concat, lpad, count

airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    agg(count(lit(1)).alias("FlightCount")). \
    show(31)

In [ ]:
# Alternative to get the count with out using agg
# We will not be able to provide alias for aggregated fields
from pyspark.sql.functions import lit, concat, lpad

airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    count(). \
    show()

### Getting total as well as delayed counts for each day

In [ ]:
from pyspark.sql.functions import lit, concat, lpad, count, sum, expr

airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    agg(count(lit(1)).alias("FlightCount"),
        sum(expr("CASE WHEN IsDepDelayed = 'YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed = 'YES' THEN 1 ELSE 0 END")).alias("ArrDelayedCount")
       ). \
    show()

### Sorting Data By FlightDate

In [ ]:
help(airlines.sort)

In [ ]:
help(airlines.orderBy)

In [ ]:
from pyspark.sql.functions import lit, concat, lpad, sum, expr
airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    agg(count(lit(1)).alias("FlightCount"),
        sum(expr("CASE WHEN IsDepDelayed = 'YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed = 'YES' THEN 1 ELSE 0 END")).alias("ArrDelayedCount")
       ). \
    orderBy("FlightDate"). \
    show(31)

### Sorting Data in descending order by count

In [ ]:
from pyspark.sql.functions import lit, concat, lpad, sum, expr, col
airlines. \
    groupBy(concat("Year", lit("-"), 
                   lpad("Month", 2, "0"), lit("-"), 
                   lpad("DayOfMonth", 2, "0")).
            alias("FlightDate")). \
    agg(count(lit(1)).alias("FlightCount"),
        sum(expr("CASE WHEN IsDepDelayed = 'YES' THEN 1 ELSE 0 END")).alias("DepDelayedCount"),
        sum(expr("CASE WHEN IsArrDelayed = 'YES' THEN 1 ELSE 0 END")).alias("ArrDelayedCount")
       ). \
    orderBy(col("FlightCount").desc()). \
    show()

## Solutions - Problem 3
Get all the flights which are departed late but arrived early (**IsArrDelayed is NO**).
* Output should contain - **FlightCRSDepTime**, **UniqueCarrier**, **FlightNum**, **Origin**, **Dest**, **DepDelay**, **ArrDelay**
* **FlightCRSDepTime** need to be computed using **Year**, **Month**, **DayOfMonth**, **CRSDepTime**
* **FlightCRSDepTime** should be displayed using **YYYY-MM-dd HH:mm** format.
* Output should be sorted by **FlightCRSDepTime** and then by the difference between **DepDelay** and **ArrDelay**
* Also get the count of such flights


In [ ]:
airlines.select('Year', 'Month', 'DayOfMonth', 'CRSDepTime').show()

In [ ]:
l = [(2008, 1, 23, 700),
     (2008, 1, 10, 1855),
    ]

In [ ]:
df = spark.createDataFrame(l, "Year INT, Month INT, DayOfMonth INT, DepTime INT")
df.show()

In [ ]:
from pyspark.sql.functions import substring
df.select(substring(col('DepTime'), -2, 2)). \
    show()

In [ ]:
df.select("DepTime", date_format(lpad('DepTime', 4, "0"), 'HH:mm')).show()

In [ ]:
help(substring)

In [ ]:
df.select(substring(col('DepTime'), 1, length(col('DepTime').cast('string')))). \
    show()

In [ ]:
from pyspark.sql.functions import lit, col, concat, lpad, sum, expr

flightsFiltered = airlines. \
    filter("IsDepDelayed = 'YES' AND IsArrDelayed = 'NO'"). \
    select(concat("Year", lit("-"), 
                  lpad("Month", 2, "0"), lit("-"), 
                  lpad("DayOfMonth", 2, "0"), lit(" "),
                  lpad("CRSDepTime", 4, "0")
                 ).alias("FlightCRSDepTime"),
           "UniqueCarrier", "FlightNum", "Origin", 
           "Dest", "DepDelay", "ArrDelay"
          ). \
    orderBy("FlightCRSDepTime", col("DepDelay") - col("ArrDelay")). \
    show()

### Getting Count

In [ ]:
from pyspark.sql.functions import lit, col, concat, lpad, sum, expr

flightsFiltered = airlines. \
    filter("IsDepDelayed = 'YES' AND IsArrDelayed = 'NO'"). \
    select(concat("Year", lit("-"), 
                  lpad("Month", 2, "0"), lit("-"), 
                  lpad("DayOfMonth", 2, "0"), lit(" "),
                  lpad("CRSDepTime", 4, "0")
                 ).alias("FlightCRSDepTime"),
           "UniqueCarrier", "FlightNum", "Origin", 
           "Dest", "DepDelay", "ArrDelay"
          ). \
    count()

flightsFiltered